In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
MaxPool2D,Conv2D, AvgPool2D, SeparableConv2D, DepthwiseConv2D, BatchNormalization, Dense, Activation, Dropout, Flatten, Input
)
from keras.constraints import max_norm
import tensorflow as tf
import numpy as np
tf.keras.backend.clear_session()
tf.random.set_seed(0)
np.random.seed(0)
electrodes=14
samples=128
temporalLength=None
dropoutRate=0.25
F1=128
F2=32
conv1Filter=64
poolPad="valid"
normRate=0.25
#(x_train.shape[1],x_train.shape[2]()x_train.shape[3])
(14, 32)
input_shape=(1,14,32)
kernLength = 2
F1 = 32
D = 2
F2 = int(F1*D)
norm_rate = 0.25
dropout_rate = 0.1
num_class=128
f1_average = 'binary' if num_class == 2 else 'macro'
data_format = 'channels_first'
Chans = input_shape[1]
Samples = input_shape[2]
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

K.set_image_data_format("channels_first")
input1       = Input(shape=input_shape)

##################################################################
block1       = Conv2D(F1, (1, kernLength), padding='same',use_bias=False)(input1)
block1       = BatchNormalization()(block1)
block1       = DepthwiseConv2D((Chans, 1), use_bias=False,
                               depth_multiplier=D,
                               depthwise_constraint=max_norm(1.))(block1)
block1       = BatchNormalization()(block1)
block1       = Activation('elu')(block1)
#block1       = AveragePooling2D((1, 4))(block1)
block1       = Dropout(dropout_rate)(block1)

block2       = SeparableConv2D(F2, (1, kernLength),
                               use_bias=False, padding='same')(block1)
block2       = BatchNormalization()(block2)
block2       = Activation('elu')(block2)
#block2       = AveragePooling2D((1, 8))(block2)
block2       = Dropout(dropout_rate)(block2)

flatten      = Flatten(name='flatten')(block2)

dense        = Dense(num_class, name='dense',
                     kernel_constraint=max_norm(norm_rate))(flatten)
softmax      = Activation('elu', name='relu')(dense)
a=Model(inputs=input1,outputs=softmax)
a.summary()


input_layer=tf.keras.layers.Input(shape=(14,32))
l1=tf.keras.layers.Reshape((1,14,32))(input_layer),
eeg=a(l1)
# output=tf.keras.layers.Dense(classification_classes*8, name="dense1",activation="sigmoid")(eeg)

output=tf.keras.layers.Dense(3, name="dense",activation="softmax")(eeg)
model=Model(inputs=input_layer, outputs=output)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)


model.load_weights('model.weights.h5',skip_mismatch=True)
model.save("modelEmotiv1.keras")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 14, 32)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 14, 32)        64        
                                                                 
 batch_normalization (Batch  (None, 32, 14, 32)        128       
 Normalization)                                                  
                                                                 
 depthwise_conv2d (Depthwis  (None, 64, 1, 32)         896       
 eConv2D)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 64, 1, 32)         128       
 chNormalization)                                                
                                                             

/home/marco/.local/lib/python3.8/site-packages/keras/src/saving/saving_lib.py:513: UserWarning: Could not load weights in object <keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f7f9e07ca90>. Skipping object. Exception encountered: Layer 'conv2d' expected 1 variables, but received 0 variables during loading. Expected: ['conv2d/kernel:0']
  _load_state(
/home/marco/.local/lib/python3.8/site-packages/keras/src/saving/saving_lib.py:513: UserWarning: Could not load weights in object <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7f7f9e07ce20>. Skipping object. Exception encountered: Layer 'batch_normalization' expected 4 variables, but received 0 variables during loading. Expected: ['batch_normalization/gamma:0', 'batch_normalization/beta:0', 'batch_normalization/moving_mean:0', 'batch_normalization/moving_variance:0']
  _load_state(
/home/marco/.local/lib/python3.8/site-packages/keras/src/saving/saving_lib.py:513: UserWarning: Could not load w